# Warsztat 7 – Pandas - wprowadzenie<a id=top></a>

<font size=2>Przed pracą z notatnikiem polecam wykonać kod w ostatniej komórce (zawiera html i css), dzięki czemu całość będzie bardziej estetyczna :)</font>

<a href='#Warsztat-91---Pandas---wprowadzenie'>Warsztat 9</a>
<ul>
<li><a href='#'><span></span></a></li>
<li><a href='#'><span></span></a></li>
<li><a href='#'><span></span></a></li>
<li><a href='#'><span></span></a></li>
</ul>

<img src="http://pandas.pydata.org/_static/pandas_logo.png" width=40% height=40%>

## Pakiet Pandas

<a href='#top' style='float: right; font-size: 13px;'>Do początku</a>  

<a href='#top' style='float: right; font-size: 13px;'>Do początku</a>  

<a href='#top' style='float: right; font-size: 13px;'>Do początku</a>  

<a href='#top' style='float: right; font-size: 13px;'>Do początku</a>  

<a href='#top' style='float: right; font-size: 13px;'>Do początku</a>  

In [1]:
from IPython.core.display import HTML
import urllib
HTML(urllib.urlopen("https://raw.githubusercontent.com/mkoculak/hello_world/nowa/ipython.css").read())